In [158]:
import tensorflow
tf = tensorflow
from tensorflow import keras
import math, random
import numpy as np
import time
from Crypto.Util.number import getPrime


# thinking

Predict a 0 or 1, for a given bit, specified in the output hint. 
Example output formats: min(a,b); abs(a-b); (a+b)/2

BTW: basic terminology problem - you should say semiprime instead of coprime... wtf

### todo:
- make it easier to run experiments and tune hyperparameters
- come up with new hyperparams
- start cataloguing which hyperparams improve performance; do a search by hand
- create a curriculum for the mask_length
- optimize for higher gpu usage; either by sticking the number generation into the gpu, speeding up prime number generation by caching prime numbers, using keras.fit[use_multiprocessing], doing the data generation in a C ext instead of python, ...

In [159]:
def generate_problem(prime_length=(140,140), mask_length=(1,280), samples=50, coprime_length=280, debug=False):
    assert min(mask_length) > 0, 'must mask at least one bit which is the question'
    assert coprime_length >= max(prime_length) * 2, 'coprime length needs to fit in 2 * max(prime_length) but doesnt'
    base = 2

    min_bits = min(prime_length)
    max_bits = max(prime_length)
    
    prime_a_length = random.randint(min_bits, max_bits)
    prime_b_length = random.randint(min_bits, max_bits)
    # this is the slowest part of the code (5/6): verify by using random ints instead of primes
    prime_a = getPrime(prime_a_length)
#     prime_a = random.randint(1, 2**prime_a_length)
    prime_b = getPrime(prime_b_length)
#     prime_b = random.randint(1, 2**prime_b_length)
    prime_a, prime_b = sorted([ prime_a, prime_b ])
    
#     shitty validation set lmao
    if prime_a == 52163 and prime_b == 54577:
        prime_a, prime_b = 43889, 50923
    
    coprime = prime_a * prime_b
    
    if debug:
        print('prime_a=', prime_a, 'prime_b=', prime_b, 'coprime=', coprime)
    
    X = np.zeros(coprime_length * 2, dtype=np.int8)
    coprime_array = X[:coprime_length]
    coprime_index = 0
    while coprime > 0:
        coprime_array[coprime_index] = coprime % base
        coprime = coprime // base
        coprime_index += 1

    factor_difference_array = X[coprime_length:]
    factor_difference = abs(prime_a - prime_b)
    factor_difference_index = 0
    while factor_difference > 0:
        factor_difference_array[factor_difference_index] = factor_difference % base
        factor_difference = factor_difference // base
        factor_difference_index += 1

    for i in range(samples):
        masked_X = X.copy()
        masked_factor_difference_array = masked_X[coprime_length:]
        masked_indices = np.random.choice(coprime_length, min(random.choice(range(min(mask_length), max(mask_length) + 1)), coprime_length), replace=False)
        # for some reason the above is slow... :/
        for index in masked_indices:
            masked_factor_difference_array[index] = -1
        prediction_index = random.choice(masked_indices)
        masked_factor_difference_array[prediction_index] = -2
        # for 0 or 1 as classes; if not using sigmoid as last layer...
        # prediction = np.zeros(2, dtype=np.int8)
        # prediction[factor_difference_array[prediction_index]] = 1
        Y = factor_difference_array[prediction_index]
        yield masked_X, Y

N = 500
timer_start = time.perf_counter()
for i in range(N):
    [i for i in generate_problem()]
timer_end = time.perf_counter()
print('time per call(generate_problem)=', (timer_end-timer_start) / N)

print('example small problem:', [i for i in generate_problem(debug=True, prime_length=(8, 8), coprime_length=16, mask_length=(3,3), samples=1)][0])


time per call(generate_problem)= 0.010872716910205782
prime_a= 163 prime_b= 179 coprime= 29177
example small problem: (array([ 1,  0,  0,  1,  1,  1,  1,  1,  1,  0,  0,  0,  1,  1,  1,  0,  0,
        0,  0,  0,  1,  0,  0,  0, -2,  0, -1, -1,  0,  0,  0,  0],
      dtype=int8), 0)


In [160]:
def format_problem(X, Y):
    coprime_length = X.shape[0] // 2
    coprime_array = X[:coprime_length]
    masked_factor_difference_array = X[coprime_length:]
    coprime = 0
    for power, bit in enumerate(coprime_array):
        coprime += bit * (2 ** (power))
    print('coprime:', coprime, '\nmasked_factor_difference:', masked_factor_difference_array, '\nY:', Y)

format_problem(*[i for i in generate_problem(debug=True, prime_length=(8, 8), coprime_length=16, mask_length=(1, 1), samples=1)][0])


prime_a= 179 prime_b= 251 coprime= 44929
coprime: 44929 
masked_factor_difference: [-2  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0] 
Y: 0


In [161]:
def row_loader(pg_args={}):
    while True:
        for X, Y in generate_problem(**pg_args):
            yield X, Y
    return

N = 500
timer_start = time.perf_counter()
for i in range(N):
    next(row_loader())
timer_end = time.perf_counter()
print('time per call(row_loader)=', (timer_end-timer_start) / N)

time per call(row_loader)= 0.005229469054378569


In [162]:
def batch_loader(pg_args={}, batch_size=2000):
    X, Y = next(generate_problem(**pg_args))
    x_feature_size = X.shape[0]
    y_feature_size = 1
    while True:
        batch_x = np.zeros((batch_size, x_feature_size), dtype=np.float32)
        batch_y = np.zeros((batch_size, y_feature_size), dtype=np.float32)
        for i, (x, y) in enumerate(row_loader(pg_args)):
            if i >= batch_size:
                break
            batch_x[i,:] = x
            batch_y[i,:] = y
        yield batch_x, batch_y

N = 10
timer_start = time.perf_counter()
for i in range(N):
    next(batch_loader())
#     next(batch_loader(pg_args={'mask_length': 120})) # masking most of the output is still wayyy too slow
timer_end = time.perf_counter()
print('time per call(batch_loader)=', (timer_end-timer_start) / N)

time per call(batch_loader)= 0.45302029312588277


In [165]:
batch_size = 1000
pg_args = {
    'prime_length': (2, 16),
    'mask_length': (1, 33),
    'samples': 100,
    'coprime_length': 33,
    'debug': False
}

model = keras.Sequential()
model.add(tf.keras.Input(shape=(pg_args['coprime_length'] * 2, )))
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# TODO: make a custom metric which is accuracy over whole prime
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 1024)              68608     
_________________________________________________________________
dense_45 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_46 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 1025      
Total params: 2,168,833
Trainable params: 2,168,833
Non-trainable params: 0
_________________________________________________________________


In [166]:
# feel free to change these params
# but be careful about adjusting other pg_args; some of them would change the shape of the network!

# MEMORY USAGE = (coprime_length * 2 + 1) * batch_size + 1
# (in bytes)

batch_size = 2000
pg_args['samples'] = 20
pg_args['mask_length'] = (1, pg_args['coprime_length'])

data_loader = batch_loader(pg_args=pg_args, batch_size=batch_size)

# TODO: experiment with use_multiprocessing, workers, tf.keras.Sequence, tf.data, etc
# ESPECIALLY: multiple threads should allow faster data generation, speedup in number of threads
# assuming bottlenecked on data generation... not sure what the evidence is though
model.fit(data_loader, epochs=1000, verbose=2, steps_per_epoch=1000)
# model.fit(data_loader, epochs=1000, verbose=2, steps_per_epoch=1000, workers=0, use_multiprocessing=True)


Epoch 1/1000


KeyboardInterrupt: 

In [147]:
model.evaluate(data_loader, verbose=2, steps=100)

100/100 - 5s - loss: 0.0170 - accuracy: 0.9782


[0.016974860802292824, 0.97816002368927]

In [148]:
def example_problem(model, data_loader):
    batch_input, batch_output = next(data_loader)
    batch_prediction = model.predict(batch_input)

    example_input = batch_input[0]
    example_output = batch_output[0]
    example_prediction = batch_prediction[0]

    format_problem(example_input, example_output)
    print('\nprediction:', example_prediction)
    return

example_problem(model, data_loader)

coprime: 177.0 
masked_factor_difference: [ 0.  0. -1. -1.  1.  1.  0. -1. -2.  0.  0.  0.  0.  0.  0. -1.  0.] 
Y: [0.]

prediction: [2.6849452e-06]


In [149]:
def primes_from_coprime_difference(coprime, factor_difference):
    average_of_factors = ((factor_difference * factor_difference + 4 * coprime) ** 0.5) / 2.0
    
    prime_a = average_of_factors + factor_difference / 2.0
    prime_b = average_of_factors - factor_difference / 2.0
    
    return prime_a, prime_b

def attempt_solution(coprime, model, debug=False):
    base = 2  # previously wasnt hardcoded but now hardcoding to keep things simple
    X_len = model.layers[0].input_shape[1]
    X_len_half = X_len//2
    X = np.zeros(X_len, dtype=np.int8)
    
    coprime_array = X[:X_len_half]
    coprime_index = 0
    coprime_residue = coprime
    while coprime_residue > 0:
        coprime_array[coprime_index] = coprime_residue % base
        coprime_residue = coprime_residue // base
        coprime_index += 1
    
    factor_difference = X[X_len_half:]
    for i in range(X_len_half):
        factor_difference[i] = -1
    
    # in the future try iterating in the last 5-10 unknown bits, since it's likely faster
    # also consider sampling based off probabilities instead of going with argmax
    while list(factor_difference).count(-1) > 0:
        if debug:
            print('factor_difference=', factor_difference)
        highest_confidence = -1.0
        highest_confidence_index = None
        highest_confidence_value = None
        for i in range(X_len_half):
            if factor_difference[i] == -1:
                factor_difference[i] = -2
                Y = model.predict(np.array([X]))
#                 if debug: # crazy debugging
#                     print(X, Y)
                confidence = abs(Y - 0.5) * 2
                if confidence > highest_confidence:
                    highest_confidence = confidence
                    highest_confidence_index = i
                    highest_confidence_value = int(Y > 0.5)
                factor_difference[i] = -1
        if debug:
            print('highest_confidence=', highest_confidence)
        factor_difference[highest_confidence_index] = highest_confidence_value
    
    if debug:
        print('factor_difference=', factor_difference)

    factor_difference_array = factor_difference
    factor_difference = 0
    for power, bit in enumerate(factor_difference_array):
        factor_difference += bit * (2 ** power)
    
    if debug:
        print('factor_difference[int]=', factor_difference)
    
    prime_a, prime_b = primes_from_coprime_difference(coprime, factor_difference)
    
    return prime_a, prime_b

In [152]:
attempt_solution(52163 * 54577, model, True)

factor_difference= [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
highest_confidence= [[1.]]
factor_difference= [ 0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
highest_confidence= [[1.]]
factor_difference= [ 0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
highest_confidence= [[1.]]
factor_difference= [ 0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
highest_confidence= [[1.]]
factor_difference= [ 0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
highest_confidence= [[1.]]
factor_difference= [ 0  0  0  0 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
highest_confidence= [[1.]]
factor_difference= [ 0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
highest_confidence= [[1.]]
factor_difference= [ 0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
highest_confidence= [[1.]]
factor_difference= [ 0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1]
highest_confidence= [[1.]]
factor_difference= [ 0  0  0  0  0  0  0  0  0 -1 -1 -1 -1 -1 -1 -1 -1]
highest_confidence= [[1.]]
factor_dif

(5.744562646538029, 5.744562646538029)

In [157]:
format_problem(*[i for i in generate_problem(debug=True, prime_length=(16, 16), coprime_length=32, mask_length=(1, 1), samples=1)][0])

prime_a= 43889 prime_b= 50923 coprime= 2234959547
coprime: 2234959547 
masked_factor_difference: [ 0  1  0  1  1  1  1  0  1  1  0  1  1  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0 -2  0] 
Y: 0
